# 편의점 3사(CU, GS25, 세븐일레븐) 홈페이지에서 매장 정보 크롤링

In [1]:
from bs4 import BeautifulSoup
import time
from datetime import datetime
from selenium import webdriver
import urllib.parse
import requests
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
# 커널을 구성하다보면 에러는 아니지만, warning이 뜨는 경우가 많은데요. 요 아래 두 부분을 씀으로 해서 그런 보기싫은 부분들을 제거해줍니다.
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

**1. 셀레니움을 이용한 cu 크롤링**

In [10]:
#cu 크롤링

location_text=[]
address_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH =  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

driver = webdriver.Chrome('C:/Users/snari/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('http://cu.bgfretail.com/store/list.do?category=store')

driver.find_element_by_xpath('//*[@id="sido"]').click()
driver.find_element_by_xpath('//*[@id="sido"]/option[3]').click()
driver.find_element_by_xpath('//*[@id="contents"]/div[1]/div/div[1]/div[2]/table/tbody/tr[1]/td[2]/div').click()


for page in range(1, 510):
    driver.execute_script('newsPage(%i);' % page) #onclick메소드 바로 가져옴
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    foundStoreNames = soup.select('div.detail_store tbody td > span.name')
    foundStoreLocations = soup.select('div.detail_info > address > a')
    
    # 5개씩 데이터를 저장
    for index in range(len(foundStoreNames)):
        location_text.append(foundStoreNames[index].text)
        address_text.append(foundStoreLocations[index].text)
        

    
    
#모든 리스트 딕셔너리형태로 저장
result= {"location" : location_text , "address":address_text }  
df = pd.DataFrame(result)  #df로 변환
# 새로 만들 파일이름 지정
outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')   
    
    

**2. 셀레니움을 이용한 gs25 크롤링**

In [16]:
#gs25 크롤링

location_text=[]
address_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH =  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

driver = webdriver.Chrome('C:/Users/snari/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('http://gs25.gsretail.com/gscvs/ko/store-services/locations')

driver.find_element_by_xpath('//*[@id="stb1"]').click()
driver.find_element_by_xpath('//*[@id="stb1"]/option[2]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="searchButton"]').click()


for page in range(1, 589):
    driver.execute_script('boardViewController.getDataList(%i);' % page) #onclick 메서드 그대로 사용
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    foundStoreNames = soup.select('#storeInfoList a.st_name')
    foundStoreLocations = soup.select('#storeInfoList a.st_address')
    
    # 5개씩 데이터를 저장
    for index in range(len(foundStoreNames)):
        location_text.append(foundStoreNames[index].text)
        address_text.append(foundStoreLocations[index].text)
        

    
    
#모든 리스트 딕셔너리형태로 저장
result= {"location" : location_text , "address":address_text }  
df = pd.DataFrame(result)  #df로 변환
# 새로 만들 파일이름 지정
outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')   
    
    

**3. requests 이용한 세븐일레븐 크롤링**

In [6]:
#세븐일레븐 크롤링

location_text=[]
address_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH =  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

# requests에 사용할 url 및 헤더 설정
url = 'http://www.7-eleven.co.kr/util/storeLayerPop.asp'
headers = {
    'Referer' : 'http://www.7-eleven.co.kr/event/eventView.asp?seqNo=573', #network에 있는 그대로 가져와야함
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
inputData = {
    'storeLaySido' : '서울',
    'storeLayGu' : '강서구',
    'hiddentext' : 'none'
}#network에 있는 그대로 가져와야함

# 서울시 행정구 리스트
guList = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '중구', '은평구',
          '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '중랑구']

# 행정구 돌아가며 데이터 가져오기
for gu in guList:
    inputData['storeLayGu'] = gu
    r = requests.post(url, data=inputData, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    time.sleep(1)
    
    foundStoreNames = soup.select('div.list_stroe a > span:nth-child(1)') #몇번째를 가져와라
    foundStoreLocations = soup.select('div.list_stroe a > span:nth-child(2)')
    
    for count in range(len(foundStoreNames)):
        location_text.append(foundStoreNames[count].get_text(strip=True))
        address_text.append(foundStoreLocations[count].text.replace('\xa0', ' '))

        

#모든 리스트 딕셔너리형태로 저장
result= {"location" : location_text , "address":address_text }  
df = pd.DataFrame(result)  #df로 변환
# 새로 만들 파일이름 지정
outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')

In [2]:
CU = pd.read_csv("./CU.csv",encoding = 'EUC-KR')
GS = pd.read_csv("./GS25.csv",encoding = 'EUC-KR')
Seven = pd.read_csv("./SEVEN.csv",encoding = 'EUC-KR')

In [29]:
CU.isnull().sum()

Unnamed: 0    0
address       0
location      0
dtype: int64

In [30]:
GS.isnull().sum()

Unnamed: 0    0
address       0
location      0
dtype: int64

In [31]:
Seven.isnull().sum()

Unnamed: 0    0
address       0
location      0
dtype: int64

In [3]:
CU = CU[CU['address'] != ' ']

In [33]:
print(CU.shape)

(2497, 3)


In [4]:
# 테이블에 브랜드 명 삽입

CU['brand'] = 'CU'
CU['location'] = 'CU ' + CU['location']

GS['brand'] = 'GS25'
GS['location'] = GS['location'].str.replace('GS25', '') # GS25는 원데이터에 GS25가 많이 포함
GS['location'] = 'GS25 ' + GS['location']

Seven['brand'] = '세븐일레븐'
Seven['location'] = '세븐일레븐 ' + Seven['location']

In [5]:
stores = pd.concat([Seven, GS, CU])
stores = stores.reset_index(drop=True)
stores.tail(10)

,Unnamed: 0,address,location,brand
8715,2534,"서울특별시 중구 퇴계로 48, (남창동)",CU 회현역점,CU
8716,2535,"서울특별시 서초구 남부순환로315길 100, (서초동)",CU 효령로점,CU
8717,2536,"서울특별시 용산구 후암로 71-2, (후암동)",CU 후암미주점,CU
8718,2537,"서울특별시 용산구 후암로 1, (후암동) 1층",CU 후암아리랑점,CU
8719,2538,"서울특별시 용산구 후암로28길 49, (후암동) 1층",CU 후암제일점,CU
8720,2539,"서울특별시 용산구 후암로 42-2, (후암동) 1층",CU 후암좋아요점,CU
8721,2541,"서울특별시 동대문구 망우로16길 26, (휘경동) 1층",CU 휘경점,CU
8722,2542,"서울특별시 동대문구 한천로 264, 주공프라자 (휘경동) 1층 102호",CU 휘경주공점,CU
8723,2543,"서울특별시 동대문구 망우로12길 5, (휘경동) 1층",CU 휘경행복점,CU
8724,2544,"서울특별시 동작구 흑석로 109-2, (흑석동)",CU 흑석로점,CU


**4. 구글맵의 지오태깅 서비스로 주소를 활용하여 위도/경도 획득**

In [8]:
import googlemaps
gmaps = googlemaps.Client(key='')

def getLatitude(location):
    try:
        return gmaps.geocode(location, language='ko')[0]['geometry']['location']['lat']
    except:
        return -1 # 주소 잘못 찍힌 것 -1로 반환

def getLongitude(location):
    try:
        return gmaps.geocode(location, language='ko')[0]['geometry']['location']['lng']
    except:
        return -1

stores['latitude'] = stores['address'].apply(lambda x: getLatitude(x)) #for문보다 속도가 빠름
stores['longitude'] = stores['address'].apply(lambda x: getLongitude(x))

# 중간 저장
stores.to_csv('./stores_with_location.csv', encoding = 'EUC-KR', index=False)
stores.tail()

,Unnamed: 0,address,location,brand,latitude,longitude
8720,2539,"서울특별시 용산구 후암로 42-2, (후암동) 1층",CU 후암좋아요점,CU,37.549777,126.977553
8721,2541,"서울특별시 동대문구 망우로16길 26, (휘경동) 1층",CU 휘경점,CU,37.588974,127.061089
8722,2542,"서울특별시 동대문구 한천로 264, 주공프라자 (휘경동) 1층 102호",CU 휘경주공점,CU,37.593228,127.069433
8723,2543,"서울특별시 동대문구 망우로12길 5, (휘경동) 1층",CU 휘경행복점,CU,37.588804,127.059294
8724,2544,"서울특별시 동작구 흑석로 109-2, (흑석동)",CU 흑석로점,CU,37.508376,126.961162
